# Assignment 5: Solving a 4×4 System with HHL
This week we implement the Harrow-Hassidim-Lloyd (HHL) quantum algorithm for a simple two-qubit (4×4) linear system, check the answer against a classical solver, and reason about when HHL helps.

This assignment mirrors the Task plan style from earlier weeks so the workflow stays reproducible.

**Task plan**
1. Note why fast quantum linear solvers matter for calibration or simulation pipelines.
2. Load and normalise a 4×4 system, compute the classical reference solution, and store diagnostics.
3. Run the HHL circuit, extract the solution register, and compare it with the classical answer.
4. Discuss accuracy, scaling behaviour, and practical roadblocks of HHL today.

## Background notes
- HHL solves systems $A\vec{x} = \vec{b}$ where $A$ is Hermitian and of size $2^n \times 2^n$. The algorithm encodes $A$ in a unitary, applies phase estimation, and performs a controlled rotation to approximate $A^{-1}$.
- For our test we choose a 4×4 Hermitian matrix that is well-conditioned so the simulation focuses on algorithmic steps instead of numerical pathologies.
- We track both the raw vector error and the residual $\lVert A\vec{x}_{\text{quantum}} - \vec{b}\rVert_2$ so classical and quantum outputs are comparable.

## Task 1 · Environment setup
- Ensure the notebook kernel has the latest `qiskit` and `qiskit-aer` packages. The magic below installs or upgrades them in place.
- After running it once, document the package versions in your lab notes.

In [ ]:
# Task 1 · Environment check
%pip install --quiet "qiskit>=0.45" "qiskit-aer"  # Run once if dependencies are missing

## Task 2 · Define the linear system
- Choose a Hermitian, well-conditioned 4×4 matrix so HHL assumptions hold.
- Normalise the right-hand side vector and store the classical reference solution for later comparison.

In [ ]:
import numpy as np
import pandas as pd
from qiskit.algorithms.linear_solvers import HHL, LinearSystemProblem
from qiskit.quantum_info import Statevector
from qiskit_aer import AerSimulator
from qiskit import transpile

# Define a Hermitian 4×4 system A·x = b
A = np.array([[1.0, -0.25, 0.0, 0.0],
              [-0.25, 1.0, -0.25, 0.0],
              [0.0, -0.25, 1.0, -0.25],
              [0.0, 0.0, -0.25, 1.0]], dtype=complex)
b = np.array([1.0, 0.5, 0.5, 0.0], dtype=complex)

# Normalise right-hand side for HHL encoding
b_norm = b / np.linalg.norm(b)

# Classical reference solution
x_classical = np.linalg.solve(A, b)
x_classical_norm = x_classical / np.linalg.norm(x_classical)

system_df = pd.DataFrame({
    "component": ["x0", "x1", "x2", "x3"],
    "classical_solution": x_classical,
    "classical_normalised": x_classical_norm
})
system_df

## Task 3 · Run HHL and extract the solution
- Build the `LinearSystemProblem` with the normalised right-hand side.
- Execute HHL on a statevector simulator so we can inspect the amplitudes directly.
- Extract the solution register, renormalise it, and compare with the classical vector.

In [ ]:
def run_hhl_and_extract(A: np.ndarray, b_normalised: np.ndarray):
    """Solve A·x=b using HHL and return raw and normalised solution vectors."""
    problem = LinearSystemProblem(matrix=A, rhs=b_normalised)
    hhl_solver = HHL()
    result = hhl_solver.solve(problem)

    # Try the direct solution vector if the API exposes it
    if hasattr(result, "solution") and result.solution is not None:
        raw_solution = np.asarray(result.solution, dtype=np.complex128)
    else:
        state_obj = getattr(result, "state", None)
        if state_obj is None:
            raise RuntimeError("HHL solver did not provide a solution vector; check Qiskit version.")

        if isinstance(state_obj, Statevector):
            statevector = np.asarray(state_obj.data, dtype=np.complex128)
        else:
            # Assume we received a QuantumCircuit and simulate it to get the statevector
            simulator = AerSimulator(method="statevector")
            compiled = transpile(state_obj, simulator)
            statevector = simulator.run(compiled).result().get_statevector()

        # Extract the first 2^n amplitudes that encode the solution register
        num_solution_amplitudes = len(b_normalised)
        raw_solution = np.asarray(statevector[:num_solution_amplitudes], dtype=np.complex128)

    raw_solution = raw_solution.flatten()
    normalised_solution = raw_solution / np.linalg.norm(raw_solution)
    return raw_solution, normalised_solution, result

raw_hhl, norm_hhl, hhl_result = run_hhl_and_extract(A, b_norm)

# Align scales with the classical answer
scale_factor = (np.vdot(x_classical, norm_hhl) / np.vdot(norm_hhl, norm_hhl))
x_hhl_rescaled = scale_factor * norm_hhl

l2_error = np.linalg.norm(x_hhl_rescaled - x_classical)
relative_error = l2_error / np.linalg.norm(x_classical)
residual = np.linalg.norm(A @ x_hhl_rescaled - b)

comparison_df = pd.DataFrame({
    "component": ["x0", "x1", "x2", "x3"],
    "classical": x_classical,
    "hhl_rescaled": x_hhl_rescaled,
    "abs_error": np.abs(x_hhl_rescaled - x_classical)
})

metrics = {
    "l2_error": float(l2_error),
    "relative_error": float(relative_error),
    "residual_norm": float(residual),
    "scale_factor": complex(scale_factor)
}

comparison_df, metrics

## Task 4 · Interpret the results
- Inspect the table above to verify the quantum vector matches the classical baseline within tolerance.
- The `metrics` dictionary reports vector error, residual, and the scale factor used to match amplitudes.

## Takeaways: significance, scalability, and limitations
- **Significance:** HHL shows how quantum phase estimation can deliver the inverse action of well-behaved matrices in $\mathcal{O}(\log N)$ qubits, which is attractive for Hamiltonian simulation, differential-equation solvers, and pre-processing in quantum machine learning.
- **Scalability:** The theoretical speed-up assumes oracles that efficiently encode sparse Hermitian matrices and good condition numbers. Gate counts grow with the precision of phase estimation, so practical circuits still scale poorly once the matrix becomes dense or ill-conditioned.
- **Shortcomings:** Current hardware depth, controlled rotations, and the need to read out the full solution limit near-term value. Extracting amplitudes requires tomography, so even with perfect gates the classical post-processing can erase the asymptotic advantage.